Required Environment:  

Python: 3.9.18  
Packages:  
  > numpy, 1.19.5  
  > sklearn, 1.3.0  
  > matplotlib, 3.7.2  
  > pickle, 4.0  


In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

Preparing data

In [ ]:
health #original spectra of heathy subject <ndarray with n rows, m wavenumbers>
hcc #original spectra of HCC subject <ndarray with n rows, m wavenumbers>

train_raw_nc, test_raw_nc = train_test_split(health, 
                                               test_size=0.3, random_state=1)
train_raw_macro, test_raw_macro = train_test_split(hcc,
                                                 test_size=0.3, random_state=1)

# A custom function to average spectras
def bootStrapMean(inputData, nSampleForMean: int, outputSize, normalize=True):

    # nSampleForMean: Determine how many samples were used to be averaged
    # outputSize: Expected size of datasets after boot strapping

    from sklearn.preprocessing import  MinMaxScaler

    outputData = np.empty((outputSize, inputData.shape[1]))
    row_order = np.arange(inputData.shape[0])
    for i in outputSize:
        np.random.shuffle(row_order)
        idx = row_order[0:nSampleForMean]
        outputData[i,:] = np.average(inputData[idx,:], axis=0)
        if normalize is True:
            scaler = MinMaxScaler()
            outputData[i,:] = scaler.fit_transform(outputData[i,:].reshape(-1,1)).reshape(1,-1)

    return outputData

x_train = np.concatenate((bootStrapMean(train_raw_nc, 10, 10000), 
                          bootStrapMean(train_raw_macro, 10, 10000)),
                          axis=0)
x_test = np.concatenate((bootStrapMean(test_raw_nc, 10, 5000), 
                         bootStrapMean(test_raw_macro, 10, 5000)),
                         axis=0)
y_train = np.concatenate([np.repeat(0, 10000), np.repeat(1, 10000)])
y_test = np.concatenate([np.repeat(0, 5000), np.repeat(1, 5000)])

Compare 11 Candidate Classifiers

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

names = ["K-nearest Neighbors Classifier(K=5)", "Linear SVM", "SVM with RBF kernel",
         "Decision Tree", "Random Forest", "AdaBoost Classifier",
         "Naive Bayes Classifier", "LASSO Logistic", "Quadratic Discriminant Analysis", 
         "Gaussian Process Classifier", "Multi-layer Perceptron Classifier"]

classifiers = [
    KNeighborsClassifier(5),
    SVC(kernel="linear", random_state=1),
    SVC(kernel="rbf", gamma='auto',  random_state=1),
    DecisionTreeClassifier(random_state=1),
    RandomForestClassifier(random_state=1),
    AdaBoostClassifier(random_state=1),
    GaussianNB(),
    Lasso(random_state=1),
    QuadraticDiscriminantAnalysis(),
    GaussianProcessClassifier(random_state=1),
    MLPClassifier(hidden_layer_sizes=5, activation='relu', learning_rate='adaptive', random_state=1)
    ]

x_t, x_v, y_t, y_v = train_test_split(x_train, y_train, test_size=0.3, random_state=1)

#Excute
clf_fitted = {}
acc_v = []
auc_v = []

for name, clf in zip(names, classifiers):
    print(name)
    clf.fit(x_t, y_t)
    clf_fitted[name] = clf

    # Avoid prediction in prob, unified them into classes
    y_t_pred = clf.predict(x_t)
    y_t_pred = np.where(y_t_pred>=0.5,1,0)
    y_v_pred = clf.predict(x_v)
    y_v_pred = np.where(y_v_pred>=0.5,1,0)

    acc_v.append(accuracy_score(y_v, y_v_pred))

    auc_v.append(roc_auc_score(y_v, y_v_pred))


Tune hyperparameters of RF

In [ ]:
# n_estimators
scores = []
score_errors = []
for i in range(1,30,3):
    rfc = RandomForestClassifier(n_estimators=i, 
                                n_jobs=-1, 
                                oob_score=True,
                                random_state=4)
    cv = cross_val_score(rfc, x_train, y_train, scoring="roc_auc",cv=10, n_jobs=-1)
    score = cv.mean() 
    error = cv.std()
    scores.append(score)
    score_errors.append(error)

# max_depth
scores = []
score_errors = []
for i in range(1,30,3):
    rfc = RandomForestClassifier(n_estimators=16,
                                max_depth=i, 
                                n_jobs=-1, 
                                oob_score=True,
                                random_state=4)
    cv = cross_val_score(rfc, x_train, y_train, scoring="roc_auc",cv=10, n_jobs=-1)
    score = cv.mean() 
    error = cv.std()
    scores.append(score)
    score_errors.append(error)

# max_features
scores = []
score_errors = []
for i in range(1,30,3):
    rfc = RandomForestClassifier(n_estimators=16,
                                max_depth=10, 
                                max_features=i,
                                n_jobs=-1, 
                                random_state=4)
    cv = cross_val_score(rfc, x_train, y_train, scoring="roc_auc",cv=10, n_jobs=-1)
    score = cv.mean() 
    error = cv.std()
    scores.append(score)
    score_errors.append(error)

Train the final RF Classifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=16, 
                             max_depth=10,
                             max_features=10,
                             random_state=2)

rfc.fit(x_train, y_train)
y_train_pred = rfc.predict(x_train)
y_test_pred = rfc.predict(x_test)

trainscore = roc_auc_score(y_train, y_train_pred)
testscore = roc_auc_score(y_test, y_test_pred)

print('trainScore:{}'.format(trainscore))
print('testScore:{}'.format(testscore))

finalmodel = rfc

Draw ROC plot

In [ ]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

y_train_predp = rfc.predict_proba(x_train)[:,1]
y_test_predp = rfc.predict_proba(x_test)[:,1]

print("Train Set Metric:")
print(classification_report(y_train, y_train_pred))
print(precision_recall_fscore_support(y_train, y_train_pred, pos_label=1))
print("Test Set Metric:")
print(classification_report(y_test, y_test_pred))
print(precision_recall_fscore_support(y_test, y_test_pred, pos_label=1))

fpr, tpr, thresh = roc_curve(y_train, y_train_predp,
                            pos_label=1)

fpr2, tpr2, thresh2 = roc_curve(y_test, y_test_predp,
                                pos_label=1) 

plt.figure(figsize=(3.5,3.5))
plt.rc('font', family='Arial', size=8)

plt.plot(fpr2,tpr2,'--',color='darkgray', lw=2,
         label="ROC test  (AUC = %0.3f)" % roc_auc_score(y_test, y_test_pred))
plt.plot(fpr,tpr,'-',color='gray', lw=2,
         label="ROC train (AUC = %0.3f)" % roc_auc_score(y_train, y_train_pred))
plt.plot([0, 1], [0, 1], color="0.6", lw=1, linestyle="--")

plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")
#plt.savefig("model_roc.pdf", bbox_inches='tight')
plt.show()

Draw confusion matrix

In [ ]:
plt.rc('font', family='Arial', size=8)

fig1, ax = plt.subplots(1,2, figsize=(6,2), tight_layout=True)
ax[0].set(title = "Train (n=20000)")
ax[0] = ConfusionMatrixDisplay.from_predictions(y_train, y_train_pred, ax=ax[0], cmap="Greys")
ax[1].set(title="Test (n=10000)")
ax[1] =ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred, ax=ax[1], cmap="Greys")
#fig1.savefig(r".\model_confusionMatrix.svg", dpi=600, transparent=False)